In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def getSentimentsAndLengthFromFile(nameOfFile):
    df=pd.DataFrame()
    file = open(nameOfFile, encoding = "utf8")
    count = 0
    column_names = []
    for line in file:
        count=count+1
        if count is 1:
            column_names = line.split('\t')
            df=pd.DataFrame(columns = column_names)
        temp_line = line.rstrip().split('\t')
#         print(count,":Line is:",temp)
        df = df.append(pd.Series(temp_line ,index = column_names), ignore_index = True)
    file.close()
    return df

In [ ]:
res = getSentimentsAndLengthFromFile("data_req_analysed.txt")

In [ ]:
print(res.head())
print(res.tail())

In [ ]:
# use below only once to save to csv
# res.to_csv(r"dataset_main.csv")

In [ ]:
df=pd.read_csv("dataset_main.csv")
print(df)